In [7]:
from PyPDF2 import PdfReader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

reader = PdfReader("Receipt.pdf")
pdf_text = ""
print("Total pages=",len(reader.pages)) 
page_numbers_to_read = [0] # Specify page number

# Read the given pages
for page in page_numbers_to_read:
    page_text = reader.pages[page].extract_text()
    pdf_text += page_text

    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large").to(device)

def query_from_list(query, options, tok_len):
    t5query = f"""Question: "{query}" Context: {options}"""
    inputs = tokenizer(t5query, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=tok_len)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)


questions = [
    "What is the Invoice no?",
    "What is the Invoice issue Date?",
    "What is the Seller name?",
    "What is the Client name?",
    "What is the Client Address?",
    "What is the Seller Address?",
    "What is the Total Net worth?",
    "What is the total Growth worth amount?"
]




Total pages= 1


Downloading:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

C:\Users\hetal\anaconda3\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hetal\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  " https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations."


NameError: name 'safe_open' is not defined

In [ ]:
# Call the LLM with input data and instruction
input_data=pdf_text
results = {question:(query_from_list(question,  input_data+"", 30))  for question in questions}
print(results)